In [ ]:
!pip install deeplake[enterprise]

In [ ]:
import copy
import deeplake
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

In [ ]:
token = ""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_ds = deeplake.load('hub://um_project/art-train', token=token, read_only=True)
dev_ds = deeplake.load('hub://um_project/art-dev', token=token, read_only=True)
val_ds = deeplake.load('hub://um_project/art-val', token=token, read_only=True)
test_ds = deeplake.load('hub://um_project/art-test', token=token, read_only=True)

In [ ]:
print(f'Size of train dataset: {len(train_ds)}')
print(f'Size of dev dataset: {len(dev_ds)}')
print(f'Size of validation dataset: {len(val_ds)}')
print(f'Size of test dataset: {len(test_ds)}')

In [ ]:
classes_labels = train_ds.labels.info.class_names
num_classes = len(classes_labels)
print(f'Number of classes: {num_classes}')
for i, label in enumerate(classes_labels):
  print(f'{i}. {label}')

In [ ]:
def plot_class_distribution(ax, class_counts, class_labels, dataset_name):
    ax.bar(np.arange(len(class_labels)), class_counts, tick_label=class_labels)
    ax.set_xlabel('Class', weight='bold')
    ax.set_xticklabels(class_labels, rotation='vertical')
    ax.set_ylabel('Number of Instances', weight='bold')
    ax.set_title(f'Frequency per Class ({dataset_name})', weight='bold')

class_train_count = np.bincount(np.concatenate(train_ds.labels.numpy(aslist = True), axis=0))
class_dev_count = np.bincount(np.concatenate(dev_ds.labels.numpy(aslist = True), axis=0))
class_val_count = np.bincount(np.concatenate(val_ds.labels.numpy(aslist = True), axis=0))
class_test_count = np.bincount(np.concatenate(test_ds.labels.numpy(aslist = True), axis=0))

fig, axs = plt.subplots(2, 2, figsize=(12, 12), constrained_layout=True)

plot_class_distribution(axs[0, 0], class_train_count, classes_labels, "Train")
plot_class_distribution(axs[0, 1], class_dev_count, classes_labels, "Dev")
plot_class_distribution(axs[1, 0], class_val_count, classes_labels, "Val")
plot_class_distribution(axs[1, 1], class_test_count, classes_labels, "Test")

plt.show()

In [ ]:
# Initialize a dictionary to store images for each class
class_images = {label: [] for label in classes_labels}

# Iterate through the dataset and store 7 images for each class
for i, sample in enumerate(train_ds):
    label = sample['labels'].data()['text'][0]  # Access the first element of the list
    if len(class_images[label]) < 7:
        image_array = sample['images'].data()['value']
        class_images[label].append(image_array)
        
    # Stop iterating if we already have 7 images for each class
    if all(len(images) == 7 for images in class_images.values()):
        break


# Create a grid of subplots and display the images
num_classes = len(classes_labels)
num_images_per_class = 7
fig = plt.figure(figsize=(28, 56))

for i, class_label in enumerate(classes_labels):
    for j, image_array in enumerate(class_images[class_label]):
        ax = fig.add_subplot(num_classes, num_images_per_class, i * num_images_per_class + j + 1)
        ax.imshow(image_array)
        ax.axis('off')

        # Set the class name above the first image in each row
        if j == int(num_images_per_class / 2):
            ax.set_title(class_label, fontsize=32, ha='center', va='center', weight='bold')

plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ImageNet statistics
])

def one_hot_encode(label, num_classes):
    one_hot = torch.zeros(num_classes)
    one_hot[label] = 1
    return one_hot

batch_size = 64
num_workers = 0

In [ ]:
def create_data_loader(dataset, batch_size, num_classes, image_transform, shuffle=True, num_workers=0):
    return dataset.pytorch(
        num_workers=num_workers,
        batch_size=batch_size,
        transform={
            'images': image_transform,
            'labels': lambda label: one_hot_encode(label, num_classes),
        },
        shuffle=shuffle,
        decode_method={'images': 'pil'}
    )

# Create data loaders for different datasets
train_loader = create_data_loader(train_ds, batch_size, num_classes, image_transform)
dev_loader = create_data_loader(dev_ds, batch_size, num_classes, image_transform)
val_loader = create_data_loader(val_ds, batch_size, num_classes, image_transform)
test_loader = create_data_loader(test_ds, batch_size, num_classes, image_transform)

In [ ]:
def save_model(model, optimizer, epoch, save_path, model_name):
  # Create the save directory if it doesn't exist
  if not os.path.exists(save_path):
    os.makedirs(save_path)

  # Create the full path for the saved model
  model_file = os.path.join(save_path, f"{model_name}_epoch_{epoch}.pth")

  # Save the model and optimizer state_dicts
  torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
  }, model_file)

  print(f"Model saved: {model_file}")

In [ ]:
def load_model(model, optimizer, load_path, device):
  # Load the saved model and optimizer state_dicts
  checkpoint = torch.load(load_path)

  # Load the model and optimizer state_dicts into the model and optimizer objects
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  # Move the model to the appropriate device (GPU or CPU)
  model.to(device)

  # Set the starting epoch for the model
  start_epoch = checkpoint['epoch']

  print(f"Model loaded: {load_path}, starting from epoch {start_epoch}")

# Usage example:
#load_path = "/content/drive/MyDrive/SSN_Projekt/Saved_Models/MultiLabelCNN_epoch_1.pth"
#load_model(model, optimizer, load_path, device)

In [ ]:
def train_validate(model, train_loader, val_loader, criterion, optimizer, device, num_epochs, patience):
    
    save_path = "/content/drive/MyDrive/UM_Projekt/Saved_Models" 
    model_name = "Net"

    best_val_accuracy = 0.0
    best_model = None
    counter = 0

    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['images'].to(device), data['labels'].to(device)

            # Convert one-hot encoded labels to class indices
            labels = torch.argmax(labels, dim=1)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_epoch_loss = running_loss / (i + 1)
        train_epoch_accuracy = correct / total * 100

        # Validation
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for i, data in enumerate(val_loader, 0):
                inputs, labels = data['images'].to(device), data['labels'].to(device)

                # Convert one-hot encoded labels to class indices
                labels = torch.argmax(labels, dim=1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_epoch_loss = running_loss / (i + 1)
        val_epoch_accuracy = correct / total * 100

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_epoch_loss:.4f}, Training Accuracy: {train_epoch_accuracy:.2f}%, Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_accuracy:.2f}%")

        # Save the model after each epoch
        save_model(model, optimizer, epoch + 1, save_path, model_name)

        # Save the best model and implement early stopping
        if val_epoch_accuracy > best_val_accuracy:
            best_val_accuracy = val_epoch_accuracy
            best_model = copy.deepcopy(model.state_dict())
            counter = 0

            save_model(model, optimizer, epoch + 1, save_path, f"{model_name}_Best")
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch + 1}. Best Validation Accuracy: {best_val_accuracy:.2f}%")
                break

    # Load the best model
    model.load_state_dict(best_model)
    return model

In [ ]:
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            inputs, labels = data['images'].to(device), data['labels'].to(device)

            # Convert one-hot encoded labels to class indices
            labels = torch.argmax(labels, dim=1)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    return test_accuracy

In [ ]:
num_classes = 13
flatten_out = 4232

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv3_8 = nn.Conv2d(3, 8, 3, 1)
        self.conv8_16 = nn.Conv2d(8, 16, 3, 1)
        self.conv16_8 = nn.Conv2d(16, 8, 3, 1)
        self.batch8 = nn.BatchNorm2d(8)
        self.batch16 = nn.BatchNorm2d(16)
        self.dropout15 = nn.Dropout(0.15)
        self.dropout25 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(flatten_out, int(flatten_out/num_classes))
        self.fc2 = nn.Linear(int(flatten_out/num_classes), num_classes)

    def forward(self, x):
        x = self.conv3_8(x)
        x = self.batch8(x)
        x = self.conv8_16(x)
        x = self.batch16(x)
        x = F.max_pool2d(x, 3)
        x = self.dropout15(x)

        # print("2")
        x = self.conv16_8(x)
        x = self.batch8(x)
        x = F.max_pool2d(x, 3)
        x = self.dropout25(x)

        # print("3")
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout25(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Model is curretly running on: {device}")
model = Net().to(device=device)

# Set the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Set the number of epochs and patience
num_epochs = 5
patience = 5

# Train and validate the model with early stopping
best_model = train_validate(model, train_loader, val_loader, criterion, optimizer, device, num_epochs, patience)

In [ ]:
save_path = "/content/drive/MyDrive/UM_Projekt/Saved_Models" 
model_name = "Net"
best_epoch = 2

best_model_path = os.path.join(save_path, f"{model_name}_Best_epoch_{best_epoch}.pth")
load_model(model, optimizer, best_model_path, device)

test_accuracy = test_model(model, test_loader, device)

In [ ]:
!pip install shap

In [ ]:
import shap

batch = next(iter(test_loader))
images, _ = batch

In [ ]:
background = images[:50].to(device)
test_images = images[50:55].to(device)

e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_images)

shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.cpu().numpy(), 1, -1), 1, 2)

def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

test_numpy = normalize_data(test_numpy)

shap.image_plot(shap_numpy, test_numpy)